In [1]:
import pandas as pd
import vaex as vx
import numpy as np
from datemerge import datemerge
from hyperlink import hyperlink
from to_excel_file import to_excel_file

topic_SP_list = ['THP Trà Xanh Không Độ','THP Nước tăng Lực Number 1','THP Sữa Đậu Nành Soya','THP Trà Thanh Nhiệt Dr. Thanh','THP Trà Sữa Macchiato Không Độ','THP Number 1 Active Chanh Muối','THP Number 1 Chanh & Dâu']
topic_list = ['Tân Hiệp Phát','Ban Lãnh Đạo Mới','THP Trà Xanh Không Độ','THP Nước tăng Lực Number 1','THP Sữa Đậu Nành Soya','THP Trà Thanh Nhiệt Dr. Thanh','THP Trà Sữa Macchiato Không Độ','THP Number 1 Active Chanh Muối','THP Number 1 Chanh & Dâu']
channel_list = ['Facebook', 'Fanpage', 'Forum', 'News']
channel_list_SP = ['Facebook', 'Fanpage', 'News', 'Youtube','Forum','Tiktok','Threads']
sent_list = ['Positive', 'Neutral', 'Negative']
labels_list = ['HOẠT ĐỘNG FANPAGE','BAN LÃNH ĐẠO','SẢN PHẨM','THƯƠNG HIỆU/CÔNG TY','CHƯƠNG TRÌNH TÀI TRỢ','QUẢNG CÁO','TUYỂN DỤNG','CHIẾN DỊCH SỰ KIỆN']

CORP = pd.read_excel(r"D:\\Data Kompa\\Tân Hiệp Phát\\Monthly\\THPsep.xlsx", sheet_name='CORP') #(7SP, THP, BLD)
SP = pd.read_excel(r"D:\\Data Kompa\\Tân Hiệp Phát\\Monthly\\THPsep.xlsx", sheet_name='SP') #(7SP - chỉnh negative)

In [2]:
 slide_n_dict = {}

In [3]:
#Slide 6

channel_topic = CORP.pivot_table(index='Topic', columns='Channel', values='Id',aggfunc = 'count').fillna(0)
channel_topic['Total'] = channel_topic.sum(axis=1)
channel_topic = channel_topic.sort_values(by='Total',ascending = False).reindex(topic_list).reset_index()
if 'forum' not in channel_topic.columns:
    channel_topic['Forum'] = 0
channel_topic['Khác'] = channel_topic.drop(columns = channel_list+ ['Topic', 'Total']).sum(axis=1)
channel_topic = channel_topic[['Topic'] + channel_list + ['Khác','Total']] 
print(channel_topic)

slide_n_dict['Slide 6'] = channel_topic

Channel                           Topic  Facebook  Fanpage  Forum  News  \
0                         Tân Hiệp Phát     851.0   1172.0      0  57.0   
1                      Ban Lãnh Đạo Mới       NaN      NaN      0   NaN   
2                 THP Trà Xanh Không Độ    1836.0     18.0      0  15.0   
3            THP Nước tăng Lực Number 1     187.0     51.0      0  25.0   
4                 THP Sữa Đậu Nành Soya    1196.0     11.0      0  67.0   
5         THP Trà Thanh Nhiệt Dr. Thanh      26.0      8.0      0  17.0   
6        THP Trà Sữa Macchiato Không Độ       5.0      0.0      0   0.0   
7        THP Number 1 Active Chanh Muối       6.0      0.0      0   6.0   
8              THP Number 1 Chanh & Dâu      11.0      0.0      0   0.0   

Channel   Khác   Total  
0         91.0  2171.0  
1          0.0     NaN  
2        115.0  1984.0  
3        194.0   457.0  
4         13.0  1287.0  
5         16.0    67.0  
6          0.0     5.0  
7          1.0    13.0  
8          1.0    12.0  

In [4]:
#Slide 7

slide_n_dict['Slide 7'] = datemerge(CORP.groupby(['PublishedDate', 'Topic']).agg({'Id': 'count'}).pivot_table(index='PublishedDate', values='Id', aggfunc='sum', columns='Topic'), '2024-08-26', '2024-09-25')
print(slide_n_dict['Slide 7'])

               THP Number 1 Active Chanh Muối  THP Number 1 Chanh & Dâu  \
PublishedDate                                                             
2024-08-26                                0.0                       0.0   
2024-08-27                                0.0                       0.0   
2024-08-28                                0.0                       0.0   
2024-08-29                                0.0                       6.0   
2024-08-30                                0.0                       3.0   
2024-08-31                                0.0                       0.0   
2024-09-01                                0.0                       0.0   
2024-09-02                                0.0                       0.0   
2024-09-03                                0.0                       0.0   
2024-09-04                                0.0                       0.0   
2024-09-05                                1.0                       0.0   
2024-09-06               

In [5]:
#Silde 8

nsr_chart_df = CORP.pivot_table(index='Topic', columns='Sentiment', values='Id', aggfunc='count').fillna(0)
nsr_chart_df['NSR'] = round((nsr_chart_df['Positive'] - nsr_chart_df['Negative']) / (nsr_chart_df['Positive'] + nsr_chart_df['Negative']), 3)
nsr_chart_df = nsr_chart_df.drop(sent_list, axis=1)

buzz_vertical_df = CORP.groupby('Topic').agg({'Id': 'count'}).pivot_table(index='Topic', values='Id', aggfunc='sum')

unique_voice_vertical_df = CORP.groupby('Topic').agg({'AuthorId': 'nunique'}).pivot_table(index='Topic', values='AuthorId', aggfunc='sum')

slide_8 = pd.concat([buzz_vertical_df, nsr_chart_df, unique_voice_vertical_df], axis=1).reindex(topic_list)
print(slide_8)
slide_n_dict['Slide 8'] = slide_8


                                    Id    NSR  AuthorId
Topic                                                  
Tân Hiệp Phát                   2171.0  0.986    1420.0
Ban Lãnh Đạo Mới                   NaN    NaN       NaN
THP Trà Xanh Không Độ           1984.0  1.000     349.0
THP Nước tăng Lực Number 1       457.0  0.961     298.0
THP Sữa Đậu Nành Soya           1287.0  0.974     695.0
THP Trà Thanh Nhiệt Dr. Thanh     67.0  0.931      49.0
THP Trà Sữa Macchiato Không Độ     5.0    NaN       3.0
THP Number 1 Active Chanh Muối    13.0  1.000      11.0
THP Number 1 Chanh & Dâu          12.0    NaN      11.0


In [6]:
#Silde 9

topic_sentiment = CORP.pivot_table(index='Topic',columns = 'Sentiment', values = 'Id',aggfunc = 'count')
topic_sentiment['Total'] = topic_sentiment.sum(axis=1)
topic_sentiment = topic_sentiment.sort_values(by = 'Total', ascending = False).fillna(0).reset_index()
topic_sentiment = topic_sentiment[['Topic'] + sent_list + ['Total']]
print(topic_sentiment)

slide_n_dict['Slide 9'] = topic_sentiment

Sentiment                           Topic  Positive  Neutral  Negative   Total
0                           Tân Hiệp Phát    1806.0    352.0      13.0  2171.0
1                   THP Trà Xanh Không Độ      44.0   1940.0       0.0  1984.0
2                   THP Sữa Đậu Nành Soya      77.0   1209.0       1.0  1287.0
3              THP Nước tăng Lực Number 1      50.0    406.0       1.0   457.0
4           THP Trà Thanh Nhiệt Dr. Thanh      28.0     38.0       1.0    67.0
5          THP Number 1 Active Chanh Muối       7.0      6.0       0.0    13.0
6                THP Number 1 Chanh & Dâu       0.0     12.0       0.0    12.0
7          THP Trà Sữa Macchiato Không Độ       0.0      5.0       0.0     5.0


In [7]:
#Slide 10
df_list_CORP = []
for i in topic_list:
    brand = CORP[CORP['Topic'] == i]
    labels_df = brand.pivot_table(index='Labels1',columns='Sentiment',values='Id',aggfunc='count').reindex(labels_list).reindex(columns = sent_list)
    labels_df_pct_sent = round(labels_df.div(labels_df.sum().sum(),axis=0).fillna(0),3)
    labels_df_pct_sent.index.name = i
    print(labels_df_pct_sent)
    df_list_CORP.append(labels_df_pct_sent)
slide_n_dict['Slide 10'] = df_list_CORP

Sentiment             Positive  Neutral  Negative
Tân Hiệp Phát                                    
HOẠT ĐỘNG FANPAGE        0.651    0.080     0.000
BAN LÃNH ĐẠO             0.000    0.000     0.000
SẢN PHẨM                 0.000    0.000     0.000
THƯƠNG HIỆU/CÔNG TY      0.131    0.047     0.006
CHƯƠNG TRÌNH TÀI TRỢ     0.049    0.006     0.000
QUẢNG CÁO                0.001    0.000     0.000
TUYỂN DỤNG               0.000    0.030     0.000
CHIẾN DỊCH SỰ KIỆN       0.000    0.000     0.000
Sentiment             Positive  Neutral  Negative
Ban Lãnh Đạo Mới                                 
HOẠT ĐỘNG FANPAGE          0.0      0.0       0.0
BAN LÃNH ĐẠO               0.0      0.0       0.0
SẢN PHẨM                   0.0      0.0       0.0
THƯƠNG HIỆU/CÔNG TY        0.0      0.0       0.0
CHƯƠNG TRÌNH TÀI TRỢ       0.0      0.0       0.0
QUẢNG CÁO                  0.0      0.0       0.0
TUYỂN DỤNG                 0.0      0.0       0.0
CHIẾN DỊCH SỰ KIỆN         0.0      0.0       0.0


In [8]:
#Slide 3
slide_3 = []

#===SOV===
for i in topic_SP_list:
    CORP['Topic'] = CORP['Topic'].replace(i,'7 Sản phẩm')
SOV = CORP.groupby('Topic').agg({'Id':'count'})
slide_3.append(SOV)
print(SOV)

#===Sentiment===
sent = CORP.groupby('Sentiment').agg({'Id':'count'}).sort_values(by='Sentiment',ascending = False)
slide_3.append(sent)
print(sent)

#===Sentiment break by Topic===
topic_sent = CORP.pivot_table(index='Topic', columns='Sentiment', values='Id',aggfunc = 'count').reindex(columns= sent_list)
topic_sent = round(topic_sent.div(topic_sent.sum(axis=1),axis=0).fillna(0),3)
slide_3.append(topic_sent)
print(topic_sent)

slide_n_dict['Slide 3'] = slide_3

                 Id
Topic              
7 Sản phẩm     3825
Tân Hiệp Phát  2171
             Id
Sentiment      
Positive   2012
Neutral    3968
Negative     16
Sentiment      Positive  Neutral  Negative
Topic                                     
7 Sản phẩm        0.054    0.945     0.001
Tân Hiệp Phát     0.832    0.162     0.006


In [10]:
# Define the groups with custom names
grouped_topic_dict = {
    'THP&BDL': ['Tân Hiệp Phát', 'Ban Lãnh Đạo Mới'],
    'Trà Xanh Không Độ': ['THP Trà Xanh Không Độ'],
    'Nước tăng Lực Number 1': ['THP Nước tăng Lực Number 1'],
    'Sữa Đậu Nành Soya': ['THP Sữa Đậu Nành Soya'],
    'Trà Thanh Nhiệt Dr Thanh': ['THP Trà Thanh Nhiệt Dr. Thanh'],
    'Trà Sữa Macchiato Không Độ': ['THP Trà Sữa Macchiato Không Độ'],
    'Chanh Muối & Chanh Dâu': ['THP Number 1 Active Chanh Muối', 'THP Number 1 Chanh & Dâu']
}

# Iterate through the dictionary
for group_name, topics in grouped_topic_dict.items():
    df_list_SP = []
    SP_df_working = SP[SP['Topic'].isin(topics)]  # Filter for the topics in the current group
    if SP_df_working.empty:
        print(f"No data available for topics: {group_name}")
        slide_n_dict[group_name] = None  # Or handle it as you prefer (e.g., append an empty list)
        continue  # Skip to the next group if no data
    
    #=======Channel========
    SP_channel = SP_df_working.pivot_table(index='Channel', values='Id', aggfunc='count').reindex(channel_list_SP)
    df_list_SP.append(SP_channel)
    
    #=======Sentiment========
    SP_sentiment = SP_df_working.pivot_table(columns='Sentiment', values='Id', aggfunc='count').reindex(columns=sent_list).fillna(0)
    if SP_df_working.empty:
        continue
    else:
        SP_sentiment = round(SP_sentiment.div(SP_sentiment.sum(axis=1), axis=0), 3)
    
    df_list_SP.append(SP_sentiment)
    
    #=======Labels========
    SP_labels = SP_df_working.pivot_table(index='Labels1', columns='Sentiment', values='Id', aggfunc='count').reindex(columns=sent_list).fillna(0)
    if SP_df_working.empty:
        continue
    else:
        SP_labels = round(SP_labels.div(SP_labels.sum().sum(), axis=0).fillna(0), 3)
        SP_labels['Total'] = SP_labels.sum(axis=1)
        SP_labels = SP_labels.sort_values(by='Total', ascending=True)
      
    df_list_SP.append(SP_labels)
    
    # Save results for the group using the group name
    slide_n_dict[group_name] = df_list_SP

# Print the keys to check
print(slide_n_dict.keys())


              Id
Channel         
Facebook   851.0
Fanpage   1172.0
News        57.0
Youtube      NaN
Forum        NaN
Tiktok      87.0
Threads      4.0
              Id
Channel         
Facebook  1836.0
Fanpage     18.0
News        15.0
Youtube     29.0
Forum        NaN
Tiktok      78.0
Threads      8.0
             Id
Channel        
Facebook  187.0
Fanpage    51.0
News       25.0
Youtube     4.0
Forum       NaN
Tiktok    189.0
Threads     1.0
              Id
Channel         
Facebook  1196.0
Fanpage     11.0
News        67.0
Youtube      NaN
Forum        NaN
Tiktok      11.0
Threads      2.0
            Id
Channel       
Facebook  26.0
Fanpage    8.0
News      17.0
Youtube    2.0
Forum      NaN
Tiktok    12.0
Threads    2.0
           Id
Channel      
Facebook  5.0
Fanpage   NaN
News      NaN
Youtube   NaN
Forum     NaN
Tiktok    NaN
Threads   NaN
            Id
Channel       
Facebook  17.0
Fanpage    NaN
News       6.0
Youtube    1.0
Forum      NaN
Tiktok     1.0
Threads    NaN
d

In [11]:
to_excel_file('THPsepOutPut', slide_n_dict)